In [ ]:
import pandas as pd
import numpy as np
import datetime as dt

In [ ]:
jan = pd.read_csv("202201-divvy-tripdata.csv")
feb = pd.read_csv("202202-divvy-tripdata.csv")
mar = pd.read_csv("202203-divvy-tripdata.csv")
apr = pd.read_csv("202204-divvy-tripdata.csv")
may = pd.read_csv("202205-divvy-tripdata.csv")
jun = pd.read_csv("202206-divvy-tripdata.csv")
jul = pd.read_csv("202207-divvy-tripdata.csv")
aug = pd.read_csv("202208-divvy-tripdata.csv")
sep = pd.read_csv("202209-divvy-tripdata.csv")
oct = pd.read_csv("202210-divvy-tripdata.csv")
nov = pd.read_csv("202211-divvy-tripdata.csv")
dec = pd.read_csv("202212-divvy-tripdata.csv")

### Merging all dataframes into one dataframe

In [ ]:
data_list = [jan, feb, mar, apr, may, jun, jul, aug, sep, oct, nov, dec]
merged_df = pd.concat(data_list)

### To find the length of each ride, I converted the started_at and ended_at columns to datetime objects so I could compute their difference

In [ ]:
merged_df['started_at'] = pd.to_datetime(merged_df['started_at'])
merged_df['ended_at'] = pd.to_datetime(merged_df   ['ended_at'])
merged_df['ride_length'] = merged_df['ended_at'] - merged_df['started_at']

### To get the day of the week that each ride started

In [ ]:
merged_df["day_of_week"] = merged_df["started_at"].dt.dayofweek

### The days are returned as numbers e.g, Monday = 0, Tuesday = 1, and so on. To change it to the days proper, I created a dictionary of the days of the week and wrote a lambda function to apply it to each column

In [ ]:
import numpy as np

weekday = {
    0: 'Monday',
    1: 'Tuesday',
    2: 'Wednesday',
    3: 'Thursday',
    4: 'Friday',
    5: 'Saturday',
    6: 'Sunday'
}
merged_df['day_of_week_name'] = merged_df['day_of_week'].map(weekday).fillna(np.nan)

#### I created column for the month and year for ride from the started_at column

In [ ]:
merged_df['year'] = pd.DatetimeIndex(merged_df['started_at']).year
merged_df['month'] = pd.DatetimeIndex(merged_df['started_at']).month

### To find out the number of riders per user type

In [ ]:
total_members = merged_df['member_casual'].value_counts()
total_members

### To find out the average duration of a ride for each user type

Casual Riders

In [ ]:
casual_member = merged_df[merged_df['member_casual'] == 'casual']
avg_casual_member_ride_length = casual_member   ['ride_length'].mean()
avg_casual_member_ride_length

Memeber Riders

In [ ]:
paid_member = merged_df[merged_df['member_casual'] == 'member']
avg_paid_member_ride_length = paid_member   ['ride_length'].mean()
avg_paid_member_ride_length

### Maximum duration for casual riders

In [ ]:
max_casual_member_ride_length = casual_member['ride_length'].max()
max_casual_member_ride_length

### Maximum duration for member riders

In [ ]:
max_member_ride_length = paid_member['ride_length'].max()
max_member_ride_length

### Most frequent Ride Day for Member Riders

In [ ]:
mode_ride_day_casual_member = casual_member['day_of_week_name'].mode()
mode_ride_day_casual_member

### Most Frequent Ride Day for Casual Riders

In [ ]:
mode_ride_day_paid_member = paid_member['day_of_week_name'].mode()
mode_ride_day_paid_member

### The number of rides per day for casual riders

In [ ]:
ride_per_day_casual = casual_member.groupby('day_of_week_name')['ride_id'].count().sort_values(ascending=False)
ride_per_day_casual

### The Number of Rides per Day for Member Riders

In [ ]:
ride_per_day_paid = paid_member.groupby('day_of_week_name')['ride_id'].count().sort_values(ascending=False)
ride_per_day_paid

### To find out the average ride length per day for casual riders

In [ ]:
avg_casual_length_per_day = casual_member[['day_of_week_name', 'ride_length']]
avg_casual_length_per_day = avg_casual_length_per_day.groupby('day_of_week_name')['ride_length'].mean()

#Convert above result to dataframe
avg_casual_length_per_day_df = pd.DataFrame({"day_of_week":avg_casual_length_per_day.index,
                                "casual_ride_length_avg": avg_casual_length_per_day.values})

avg_casual_length_per_day_df['casual_ride_length_avg'] = avg_casual_length_per_day_df['casual_ride_length_avg']/pd.Timedelta(minutes=1) #To get ride length in minutes
avg_casual_length_per_day_df

### Average Ride Length per Day for Member Riders

In [ ]:
avg_paid_length_per_day = paid_member[['day_of_week_name', 'ride_length']]
avg_paid_length_per_day =   avg_paid_length_per_day.groupby   ('day_of_week_name')['ride_length'].mean()

avg_paid_length_per_day_df = pd.DataFrame   ({"day_of_week":avg_paid_length_per_day.index,
"paid_ride_length_avg": avg_paid_length_per_day.values})

avg_paid_length_per_day_df  ['paid_ride_length_avg'] =    avg_paid_length_per_day_df ['paid_ride_length_avg']/pd.Timedelta(minutes=1) #To get ride length in minutes

avg_paid_length_per_day_df

### Lastly, we will check for any monthly trends in the data.

In [ ]:
monthly_trend = merged_df.groupby("month")["member_casual"].value_counts()
monthly_trend_df = monthly_trend.to_frame()
monthly_trend_df
#Convert to dataframe then to CSV file
monthly_trend_df = monthly_trend.unstack(level=1)
monthly_trend_df